In [134]:
import pandas as pd
import numpy as np

df = pd.read_csv('newdataset.csv', low_memory=False)

In [135]:
def clean_column(column):
    return (
        column.strip(' ')
        .replace('/','_')
        .replace(' ','_')
        .lower()
    )

In [136]:
for column in df.columns:
    column = clean_column(column)

In [137]:
df = df[df['label'] != 'label']
df.shape

(1455183, 76)

### Ataki pogrupowane 

In [138]:
labels = ['benign', 'ddos', 'portscan', 'botnet - attempted', 'botnet',
          'infiltration - portscan', 'web attack - xss - attempted',
          'web attack - brute force - attempted', 'web attack - brute force',
          'infiltration - attempted', 'web attack - xss', 'infiltration',
          'web attack - sql injection', 'web attack - sql injection - attempted',
          'ftp-patator', 'ssh-patator', 'ssh-patator - attempted',
          'ftp-patator - attempted', 'dos hulk', 'dos goldeneye',
          'dos slowloris', 'dos slowhttptest - attempted',
          'dos slowhttptest', 'dos slowloris - attempted', 'heartbleed',
          'dos hulk - attempted', 'dos goldeneye - attempted']

# Funkcja przypisująca słowa kluczowe do listy label_val
def assign_label_val(label):
    keywords = ['dos', 'ftp-patator', 'ssh-patator', 'web attack', 'botnet', 'infiltration']
    for keyword in keywords:
        if keyword in label:
            if keyword == 'ftp-patator' or keyword == 'ssh-patator':
                return 'brute force'
            elif keyword == 'ddos':
                return 'dos'
            else:
                return keyword
    return label
    
df['label_group'] = df['label'].apply(assign_label_val)

print(df['label_group'].unique())

labels_group = df['label_group'].unique()
label_mapping = {'benign': 0}
label_mapping.update({label: idx + 10 for idx, label in enumerate(labels_group) if label != 'benign'})
df['label_val'] = df['label_group'].map(label_mapping)

print(df[['label_group', 'label_val']].head())

['benign' 'dos' 'portscan' 'botnet' 'infiltration' 'web attack'
 'brute force' 'heartbleed']
  label_group  label_val
0      benign          0
1         dos         11
2    portscan         12
3    portscan         12
4    portscan         12


In [139]:
print(df['label_val'].unique())

[ 0 11 12 13 14 15 16 17]


In [140]:
df.shape

(1455183, 78)

In [141]:
df.head()

,fname,flow_duration,flow_bytes_s,flow_packets_s,fwd_packets_s,bwd_packets_s,packet_length_min,packet_length_max,packet_length_mean,average_packet_size,...,bwd_iat_min,bwd_segment_size_avg,bwd_psh_flags,bwd_rst_flags,bwd_header_length,bwd_init_win_bytes,subflow_bwd_bytes,label,label_group,label_val
0,/mnt/ddata/shared/anly/pcaps/out/friday/tcp-00...,5279730.000000,0.000000,1.136422,0.757614,0.378807,0.000000,0.000000,0.000000,0.000000,...,5219097.000000,0.000000,0.000000,0.000000,52.000000,28400.000000,0.000000,benign,benign,0
1,/mnt/ddata/shared/anly/pcaps/out/friday/tcp-00...,9575072.000000,1213.045654,1.148816,0.731065,0.417751,0.000000,11595.000000,1055.909058,1055.909058,...,6.000000,2898.750000,1.000000,0.000000,92.000000,29200.000000,11595.000000,ddos,dos,11
2,/mnt/ddata/shared/anly/pcaps/out/friday/tcp-00...,53.000000,0.000000,37735.847656,18867.923828,18867.923828,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,20.000000,0.000000,0.000000,portscan,portscan,12
3,/mnt/ddata/shared/anly/pcaps/out/friday/tcp-00...,15.000000,0.000000,133333.328125,66666.664062,66666.664062,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,20.000000,0.000000,0.000000,portscan,portscan,12
4,/mnt/ddata/shared/anly/pcaps/out/friday/tcp-00...,76.000000,0.000000,26315.789062,13157.894531,13157.894531,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,20.000000,0.000000,0.000000,portscan,portscan,12


In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1455183 entries, 0 to 1455186
Data columns (total 78 columns):
 #   Column                      Non-Null Count    Dtype 
---  ------                      --------------    ----- 
 0   fname                       1455183 non-null  object
 1   flow_duration               1455183 non-null  object
 2   flow_bytes_s                1455183 non-null  object
 3   flow_packets_s              1455183 non-null  object
 4   fwd_packets_s               1455183 non-null  object
 5   bwd_packets_s               1455183 non-null  object
 6   packet_length_min           1455183 non-null  object
 7   packet_length_max           1455183 non-null  object
 8   packet_length_mean          1455183 non-null  object
 9   average_packet_size         1455183 non-null  object
 10  packet_length_std           1455183 non-null  object
 11  packet_length_variance      1455183 non-null  object
 12  flow_iat_mean               1455183 non-null  object
 13  flow_iat_std

### Ataki nie pogrupowane 

In [45]:
labels = ['benign', 'ddos', 'portscan', 'botnet - attempted', 'botnet',
          'infiltration - portscan', 'web attack - xss - attempted',
          'web attack - brute force - attempted', 'web attack - brute force',
          'infiltration - attempted', 'web attack - xss', 'infiltration',
          'web attack - sql injection', 'web attack - sql injection - attempted',
          'ftp-patator', 'ssh-patator', 'ssh-patator - attempted',
          'ftp-patator - attempted', 'dos hulk', 'dos goldeneye',
          'dos slowloris', 'dos slowhttptest - attempted',
          'dos slowhttptest', 'dos slowloris - attempted', 'heartbleed',
          'dos hulk - attempted', 'dos goldeneye - attempted']

label_mapping = {'benign': 0}
label_mapping.update({label: idx + 10 for idx, label in enumerate(labels) if label != 'benign'})
df['label_val'] = df['label'].map(label_mapping)

print(df[['label', 'label_val']].head())


      label  label_val
0    benign          0
1      ddos         11
2  portscan         12
3  portscan         12
4  portscan         12


In [144]:
df = df.drop(columns=['fname','label'])

In [146]:
df.rename(columns={'label_group': 'label'}, inplace=True)
columns_to_convert = df.columns.difference(['label', 'label_val'])
df[columns_to_convert] = df[columns_to_convert].astype(float)
print(df.dtypes)
df.to_csv('data_tcpudp_with_labels.csv', index=False)

flow_duration         float64
flow_bytes_s          float64
flow_packets_s        float64
fwd_packets_s         float64
bwd_packets_s         float64
                       ...   
bwd_header_length     float64
bwd_init_win_bytes    float64
subflow_bwd_bytes     float64
label                  object
label_val               int64
Length: 76, dtype: object


In [112]:
X = df[columns_to_convert]
#y = df['label_val']
y = df['label_group_val']

In [113]:
y.value_counts()

0     976917
11    244730
12    158750
14     60944
16      6972
13      4803
15      2056
17        11
Name: label_group_val, dtype: int64

In [70]:
X.describe()

,ack_flag_count,active_max,active_mean,active_min,active_std,average_packet_size,bwd_bulk_rate_avg,bwd_bytes_bulk_avg,bwd_header_length,bwd_iat_max,...,psh_flag_count,rst_flag_count,subflow_bwd_bytes,subflow_fwd_bytes,subflow_fwd_packets,syn_flag_count,total_bwd_packets,total_fwd_packet,total_length_of_bwd_packet,total_length_of_fwd_packet
count,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06,...,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06,1.455183e+06
mean,3.446769e+01,2.849863e+05,1.528539e+05,1.124441e+05,7.518795e+04,2.601410e+02,3.336573e+06,5.678067e+03,4.341436e+02,4.463907e+06,...,4.467475e+00,5.832077e-01,7.721176e+03,2.391125e+03,5.239170e+00,1.282223e+00,1.863070e+01,1.800304e+01,2.225452e+04,9.868388e+03
std,2.436025e+03,1.283525e+06,8.325771e+05,7.550082e+05,4.963662e+05,3.287886e+02,1.107914e+07,3.629039e+05,2.482458e+04,1.236147e+07,...,3.368429e+01,1.052516e+00,1.395804e+06,8.651319e+05,5.699252e+02,9.960687e-01,1.217109e+03,1.243690e+03,2.272185e+06,2.190873e+06
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.800000e+01,0.000000e+00,0.000000e+00,1.600000e+01,3.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,2.000000e+00,9.800000e+01,2.000000e+01
50%,9.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.950000e+01,0.000000e+00,0.000000e+00,9.600000e+01,4.508800e+04,...,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,4.000000e+00,6.000000e+00,3.700000e+02,9.400000e+01
75%,1.700000e+01,3.926300e+04,3.293000e+04,2.292500e+04,0.000000e+00,3.309278e+02,0.000000e+00,0.000000e+00,2.320000e+02,4.999542e+06,...,5.000000e+00,1.000000e+00,2.940000e+02,3.933333e+01,3.500000e+00,2.000000e+00,8.000000e+00,1.000000e+01,6.847000e+03,5.800000e+02
max,5.116810e+05,1.100975e+08,1.100975e+08,1.100975e+08,7.415440e+07,2.125667e+03,1.369000e+09,1.567619e+08,5.838440e+06,1.199914e+08,...,9.861000e+03,1.900000e+02,6.270479e+08,5.905969e+08,2.706870e+05,5.200000e+01,2.919220e+05,2.895860e+05,6.554524e+08,6.396514e+08


### Test datasets 

In [109]:
df_test = df.sample(n=500000, random_state=42)
X = df_test[columns_to_convert]
#y = df_test['label_val']
y = df_test['label_group_val']
y.unique()

array([ 0, 12, 11, 14, 16, 13, 15, 17])

## Models RF + GB

In [78]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, ClassifierMixin

# Definicja ConditionalClassifier
class ConditionalClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, binary_clf, multiclass_clf):
        self.binary_clf = binary_clf
        self.multiclass_clf = multiclass_clf

    def fit(self, X, y):
        self.binary_clf.fit(X, y)
        binary_preds = self.binary_clf.predict(X)
        X_multiclass = X[binary_preds != 0]
        y_multiclass = y[binary_preds != 0]
        self.multiclass_clf.fit(X_multiclass, y_multiclass)
        
        return self

    def predict(self, X):
        binary_preds = self.binary_clf.predict(X)
        final_preds = np.copy(binary_preds)
        X_multiclass = X[binary_preds != 0]
        multiclass_preds = self.multiclass_clf.predict(X_multiclass)
        final_preds[binary_preds != 0] = multiclass_preds
        
        return final_preds


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

binary_clf = RandomForestClassifier(
    max_depth=20,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=500,
    random_state=42
)

multiclass_clf = GradientBoostingClassifier(n_estimators=100)  

conditional_clf = ConditionalClassifier(binary_clf, multiclass_clf)

pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('clf', conditional_clf)       
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
conditional_accuracy = accuracy_score(y_test, y_pred)

cf_matrix = confusion_matrix(y_test, y_pred)
cfm = cf_matrix / np.sum(cf_matrix, axis=1)[:, None]
df_cm = pd.DataFrame(cfm, index=np.unique(y), columns=np.unique(y))

with np.printoptions(precision=3, suppress=True):
    print('cm_diag', np.diag(df_cm))

plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, fmt='.2f', cbar=True, 
            xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix Heatmap for Conditional Classifier')
plt.show()


KeyboardInterrupt: 

## Models GAN + GB

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Definicja Discriminatora
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

# Definicja ConditionalClassifier
class ConditionalClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, binary_clf, multiclass_clf, scaler):
        self.binary_clf = binary_clf
        self.multiclass_clf = multiclass_clf
        self.scaler = scaler
        
    def fit(self, X, y):
        X = self.scaler.transform(X)
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor((y > 0).astype(int).values, dtype=torch.float32).view(-1, 1)
        self.binary_clf.train()
        criterion = nn.BCELoss()
        optimizer = optim.Adam(self.binary_clf.parameters(), lr=0.0001)
        epochs = 500

        for epoch in range(epochs):
            outputs = self.binary_clf(X_tensor)
            loss = criterion(outputs, y_tensor)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 50 == 0:
                print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

        with torch.no_grad():
            binary_preds = (self.binary_clf(X_tensor).numpy() > 0.5).astype(int).flatten()
            
        mask = binary_preds != 0
        X_multiclass = X[binary_preds != 0,:]
        y_multiclass = y[binary_preds != 0]
        self.multiclass_clf.fit(X_multiclass, y_multiclass)
        
        return self

    def predict(self, X):
        X = self.scaler.transform(X)
        X_tensor = torch.tensor(X, dtype=torch.float32)
        
        with torch.no_grad():
            binary_preds = (self.binary_clf(X_tensor).numpy() > 0.5).astype(int).flatten()
        
        X_multiclass = X[binary_preds != 0]
        multiclass_preds = self.multiclass_clf.predict(X_multiclass)
        final_preds = np.copy(binary_preds)
        final_preds[binary_preds != 0] = multiclass_preds       
        return final_preds 
    

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
scaler.fit(X_train)

input_dim = X_train.shape[1]
discriminator = Discriminator(input_dim)
multiclass_clf = GradientBoostingClassifier(n_estimators=100)

conditional_clf = ConditionalClassifier(discriminator, multiclass_clf, scaler)

conditional_clf.fit(X_train, y_train)
y_pred = conditional_clf.predict(X_test)

conditional_accuracy = accuracy_score(y_test, y_pred)
print("Conditional Classifier Accuracy:", conditional_accuracy)



Epoch [50/500], Loss: 0.6558
Epoch [100/500], Loss: 0.5922
Epoch [150/500], Loss: 0.4731
Epoch [200/500], Loss: 0.3441
Epoch [250/500], Loss: 0.2314
Epoch [300/500], Loss: 0.1612
Epoch [350/500], Loss: 0.1240
Epoch [400/500], Loss: 0.1037
Epoch [450/500], Loss: 0.0909
Epoch [500/500], Loss: 0.0816


In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred)
cfm = cf_matrix / np.sum(cf_matrix, axis=1)[:, None]
df_cm = pd.DataFrame(cfm, index=np.unique(y_test), columns=np.unique(y_test))

with np.printoptions(precision=3, suppress=True):
    print('cm_diag', np.diag(df_cm))

plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, fmt='.2f', cbar=True, 
            xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix Heatmap for Conditional Classifier')
plt.show()

In [ ]:
'benign' 'dos' 'portscan' 'botnet' 'infiltration' 'web attack'
 'brute force' 'heartbleed'